In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#Load the saved model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BI_90_in_24mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [3]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [4]:
combined_data = model_data.copy()

In [5]:
#Make sure select correct features or not
loaded_model.feature_names

['Num_Enq_non_BANK_last_12M',
 'Num_Enq_all_loans_last_9M_3M',
 'No of enq in past 90days',
 'num_PERSONAL LOAN_enq_last_36M',
 'Num_of_PERSONAL LOAN_opened_last9M',
 'Num_of_PERSONAL LOAN_opened_last12M',
 'highest_sanction_amount_credit_cards',
 'num_CONSUMER LOAN_enq_last_36M',
 'num_PL_LE_1_Lac_opened_in_last_12M',
 'Total_outstanding_Balance_PERSONAL LOAN_opened_last_6M',
 'Utilization_PERSONAL LOAN_opened_last_12M',
 'Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M',
 'Utilization_all',
 'Age_of_oldest_open_account',
 'total_emi',
 'percentage_ever_delinquent',
 'num_CONSUMER LOAN_enq_last_6M_36M',
 'bankcard_vel_2',
 'g406s']

In [6]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [7]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [8]:
combined_df= combined_data.drop_duplicates(subset='unique_reference_number')

In [9]:
combined_df= combined_df.drop_duplicates(subset='latest_feature_id')

In [10]:
# if vintage is from greater than equal to oct-22 use 60-15 bad def else use 90-24 bad def for get gini ks of that particular vintage
df1 = combined_df.dropna(subset=['60_in_15mob'])

In [11]:
df1.shape

(123367, 2831)

In [13]:
df = df1.copy()

In [14]:
df.shape

(123367, 2831)

In [15]:
df1['unique_reference_number'].nunique()

123367

In [16]:
#change the month-year (Various Vintage)
df = df[df['month_year']=='2023-04']

In [17]:
df.shape

(6050, 2831)

In [18]:
data = df.copy()

In [19]:
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_3026282/499432767.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [20]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [21]:
model_df1.shape, oot_df1.shape

((0, 2831), (6050, 2831))

In [24]:
features_list=[
'Num_Enq_non_BANK_last_12M',
 'Num_Enq_all_loans_last_9M_3M',
 'No of enq in past 90days',
 'num_PERSONAL LOAN_enq_last_36M',
 'Num_of_PERSONAL LOAN_opened_last9M',
 'Num_of_PERSONAL LOAN_opened_last12M',
 'highest_sanction_amount_credit_cards',
 'num_CONSUMER LOAN_enq_last_36M',
 'num_PL_LE_1_Lac_opened_in_last_12M',
 'Total_outstanding_Balance_PERSONAL LOAN_opened_last_6M',
 'Utilization_PERSONAL LOAN_opened_last_12M',
 'Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M',
 'Utilization_all',
 'Age_of_oldest_open_account',
 'total_emi',
 'percentage_ever_delinquent',
 'num_CONSUMER LOAN_enq_last_6M_36M',
 'bankcard_vel_2',
 'g406s',
    'month_year',
 '60_in_15mob']

In [25]:
model_df = data[features_list]

In [26]:
model_df.shape

(6050, 21)

In [27]:
model_df = model_df1[features_list]

In [28]:
oot_df = oot_df1[features_list]

In [29]:
model_df.shape, oot_df.shape

((0, 21), (6050, 21))

In [30]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [31]:
X_model.shape

(0, 19)

In [32]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [33]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3026282/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [34]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3026282/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [35]:
X_oot.shape,y_oot.shape

((6050, 19), (6050,))

In [39]:
# # Create DMatrix for OOT predictions
# if records available on OOT split use this cell else use below cell
data_x_pred = xgb.DMatrix(X_oot)
X_oot['60_in_15mob'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [36]:
# Create DMatrix for OOT predictions
data_x_pred = xgb.DMatrix(X_model)
X_model['60_in_15mob'] = y_model
X_model['month_year'] = model_df.loc[model_df.index,'month_year'].values

X_model['y_pred'] = loaded_model.predict(data_x_pred)

/home/jayasurya.vijayakumar/miniconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:22:17] WARNING: /workspace/src/common/error_msg.cc:52: Empty dataset at worker: 0
  warnings.warn(smsg, UserWarning)


In [40]:

summary,auc, gini, ks = decile_analysis(X_oot, 'y_pred', '60_in_15mob')
# summary,auc, gini, ks = decile_analysis(X_model, 'y_pred', '60_in_15mob')


In [42]:
# print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
# print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"AUC: {auc:.2f}, Gini: {gini:.2f}, KS: {ks:.2f}")

AUC: 0.69, Gini: 0.37, KS: 0.29


In [44]:
summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.238017         0.335834        0.549828        0.273124    605   
9          0.178512         0.244540        0.273087        0.222647    605   
8          0.145455         0.204284        0.222623        0.188982    605   
7          0.119008         0.175768        0.188831        0.164171    605   
6          0.100826         0.154965        0.164136        0.146898    605   
5          0.071906         0.140482        0.146824        0.134502    598   
4          0.055556         0.129229        0.134490        0.123890    612   
3          0.061462         0.118105        0.123871        0.112508    602   
2          0.044408         0.108657        0.112498        0.102144    608   
1          0.031405         0.091187        0.102128        0.070692    605   

        label_good  label_bad  
Decile                         
10             461        144  
9              497        108  
8              517         88  
7              533         72  
6              544         61  
5              555         43  
4              578         34  
3              565         37  
2              581         27  
1              586         19